In [ ]:
!pip install transformers lightning pytorch-lightning lightning-bolts -qq

## Generating data to overfit LongT5

In [ ]:
import pandas as pd
df = pd.read_csv("longt5_dataset_1.tsv", header=None, sep="\t")

In [ ]:
import random

def get_sample_pair(prompt, target, n_samples):
  shuffled_samples = []
  splitted_target = target.split(">")
  for i in range(n_samples):
    shuffled_samples.append([int(splitted_target[i].strip()[1]), i])
  splitted_prompt = prompt.split("\n")

  new_prompt = splitted_prompt[:2]
  random.shuffle(shuffled_samples)

  new_idxs_ranking = [0] * n_samples
  for idx, sample in enumerate(shuffled_samples):
    doc, rank = sample
    splitted_prompt[doc + 1] = f"[doc_{idx + 1}] {splitted_prompt[doc + 1][3:]}"
    new_prompt.append(splitted_prompt[doc + 1])
    new_idxs_ranking[rank] = f"doc_{idx + 1}"
  new_prompt.extend(splitted_prompt[-3:])
  target = ""
  for rank in new_idxs_ranking:
    target += f"[{rank}] > "
  output = ["\n".join(new_prompt), target[:-3]]

  output[0] = output[0].replace("10 passages", f"{n_samples} passages")
  return output

In [ ]:
overfit_data = []
for i in range(2, 10):
  text_prompts = []
  targets = []
  for idx, row in df.iterrows():
    try:
      text_prompt, target = get_sample_pair(row[0], row[1], i)
      text_prompts.append(text_prompt)
      targets.append(target)
    except:
      continue
  overfit_data.append([text_prompts, targets, i])

In [ ]:
for text_prompts, targets, n_samples in overfit_data:
  df_tmp = pd.DataFrame()
  df_tmp["prompt"] = text_prompts
  df_tmp["target"] = targets
  df_tmp.to_csv(f"longt5_data_{n_samples}_samples.tsv", sep="\t", index=False)

## Random dataset

In [ ]:
text_prompts = [
    "a b x z",
    "l x 8 1",
    "p u 7 0",
    "j y 6 4",
    "m g 5 e",
    "p k 8 9",
    "l m j y 6",
    "t p k 1"
]

targets = [
    "a b x z",
    "l x 8 1",
    "p u 7 0",
    "j y 6 4",
    "m g 5 e",
    "p k 8 9",
    "l m j y 6",
    "t p k 1"
]

df_random = pd.DataFrame()
df_random["prompt"] = text_prompts
df_random["target"] = targets
df_random.to_csv(f"longt5_data_random_samples.tsv", sep="\t", index=False)

## Overfitting test using different window sizes

In [ ]:
!python train.py \
 --pretrained_model_name_or_path google/long-t5-tglobal-base \
 --overfit_test \
 --data_path /content/longt5_data_random_samples.tsv

2023-06-28 04:20:05.062606: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
---------------------------------------------------------
0 | model | LongT5ForConditionalGeneration | 247 M 
---------------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
990.350   Total estimated model params size (MB)
Sanity Checking DataLoader 0:   0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
a b x z a b x z a b x z a b x z a b x z a b x z a b x z a b x z a b x z a b x z a b x z a

In [ ]:
!python train.py \
 --pretrained_model_name_or_path google/long-t5-tglobal-base \
 --overfit_test \
 --data_path /content/longt5_data_2_samples.tsv

2023-06-28 04:22:48.275432: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
---------------------------------------------------------
0 | model | LongT5ForConditionalGeneration | 247 M 
---------------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
990.350   Total estimated model params size (MB)
Sanity Checking DataLoader 0:   0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
I can rank them based on their relevance to query: how do i fill in a tile floor divot [d

In [ ]:
!python train.py \
 --pretrained_model_name_or_path google/long-t5-tglobal-base \
 --overfit_test \
 --data_path /content/longt5_data_3_samples.tsv

2023-06-28 04:26:00.294997: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
---------------------------------------------------------
0 | model | LongT5ForConditionalGeneration | 247 M 
---------------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
990.350   Total estimated model params size (MB)
Sanity Checking DataLoader 0:   0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
I can rank them based on their relevance to query: how do i fill in a tile floor divot [d

In [ ]:
!python train.py \
 --pretrained_model_name_or_path google/long-t5-tglobal-base \
 --overfit_test \
 --data_path /content/longt5_data_4_samples.tsv

2023-06-28 04:30:36.136512: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
---------------------------------------------------------
0 | model | LongT5ForConditionalGeneration | 247 M 
---------------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
990.350   Total estimated model params size (MB)
Sanity Checking DataLoader 0:   0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
I can rank them based on their relevance to query: how do i fill in a tile floor divot [d

In [ ]:
!python train.py \
 --pretrained_model_name_or_path google/long-t5-tglobal-base \
 --overfit_test \
 --data_path /content/longt5_data_5_samples.tsv

2023-06-28 04:35:57.250383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
---------------------------------------------------------
0 | model | LongT5ForConditionalGeneration | 247 M 
---------------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
990.350   Total estimated model params size (MB)
Sanity Checking DataLoader 0:   0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
I can rank them based on their relevance to query: how do i fill in a tile floor divot [d

In [ ]:
!python train.py \
 --pretrained_model_name_or_path google/long-t5-tglobal-base \
 --overfit_test \
 --data_path /content/longt5_data_6_samples.tsv

2023-06-28 04:43:49.355682: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
---------------------------------------------------------
0 | model | LongT5ForConditionalGeneration | 247 M 
---------------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
990.350   Total estimated model params size (MB)
Sanity Checking DataLoader 0:   0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
I would like to insert a parquet floor design into my kitchen floor which is tiled with i

In [ ]:
!python train.py \
 --pretrained_model_name_or_path google/long-t5-tglobal-base \
 --overfit_test \
 --data_path /content/longt5_data_7_samples.tsv

2023-06-28 04:54:45.621093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
---------------------------------------------------------
0 | model | LongT5ForConditionalGeneration | 247 M 
---------------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
990.350   Total estimated model params size (MB)
Sanity Checking DataLoader 0:   0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
I can rank them based on their relevance to query: how do i fill in a tile floor divot [d

In [ ]:
!python train.py \
 --pretrained_model_name_or_path google/long-t5-tglobal-base \
 --overfit_test \
 --data_path /content/longt5_data_8_samples.tsv

2023-06-28 05:09:39.365442: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
---------------------------------------------------------
0 | model | LongT5ForConditionalGeneration | 247 M 
---------------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
990.350   Total estimated model params size (MB)
Sanity Checking DataLoader 0:   0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
I can rank them based on their relevance to query: how do i fill in a tile floor divot [d

In [ ]:
!python train.py \
 --pretrained_model_name_or_path google/long-t5-tglobal-base \
 --overfit_test \
 --data_path /content/longt5_data_9_samples.tsv

2023-06-28 05:25:15.404176: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
---------------------------------------------------------
0 | model | LongT5ForConditionalGeneration | 247 M 
---------------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
990.350   Total estimated model params size (MB)
Sanity Checking DataLoader 0:   0% 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
I can rank them based on their relevance to query: how do i fill in a tile floor divot [d